In [ ]:
!pip3 install ../input/packages/dicom2nifti-2.3.0/dicom2nifti-2.3.0
!pip3 install ../input/packages/fslpy-03.7.0-py2.py3-none-any.whl
!pip3 install ../input/packages/isodate-0.6.0-py2.py3-none-any.whl
!pip3 install ../input/packages/rdflib-6.0.1-py3-none-any.whl
!pip3 install ../input/packages/prov-2.0.0-py3-none-any.whl
!pip3 install ../input/packages/ci_info-0.2.0-py3-none-any.whl
!pip3 install ../input/packages/etelemetry-0.2.2-py3-none-any.whl
!pip3 install ../input/packages/traits-6.2.0/traits-6.2.0.tar
!pip3 install ../input/packages/nipype-1.6.1-py3-none-any.whl

In [ ]:
# import shutil
# shutil.rmtree("./Final_test")
# shutil.rmtree("./step1_test")

In [ ]:
!python ../input/pytorch-timm/preprocess4.py

In [ ]:
# import timm 
import torch
import os
import shutil
import pydicom
import numpy as np
import cv2
import random
import nibabel as nib
from nibabel.viewers import OrthoSlicer3D
# import fsl
# import dicom2nifti
import numpy as np 
import pandas as pd

### 生成测试用的2d数据

In [ ]:
# preserve_ratio=0.2
# scan_type_list=["FLAIR"]
# # scan_type_list=["FLAIR,T1w,T1wCE,T2w"]

# data_dir="/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/test"
# patient_no_test=os.listdir(data_dir)
# # patient_no_test=["00001"]
# print("test numbers:",len(patient_no_test))

# width_list=[]
# height_list=[]

# output_dir_test="/kaggle/working/flair2d/test/0"
# if not os.path.exists(output_dir_test):
#     os.makedirs(output_dir_test)
# output_dir_test2="/kaggle/working/flair2d/test/1"
# if not os.path.exists(output_dir_test2):
#     os.makedirs(output_dir_test2)

# for patient_no in patient_no_test:
#     flag=0
#     for scan_type in scan_type_list:
#         image_dir=os.path.join(data_dir,patient_no,scan_type)
#         dicom_list=os.listdir(image_dir)
#         for dicom_name in dicom_list:
#             dicom_path=os.path.join(image_dir,dicom_name)
#             dicom=pydicom.read_file(dicom_path)
#             image=dicom.pixel_array
#             max_pixel=np.amax(image)
#             if max_pixel >0:
#                 image=image/max_pixel*255
#                 if float(np.count_nonzero(image))/image.size>=preserve_ratio:
#                     width, height=image.shape
#                     width_list.append(width)
#                     height_list.append(height)
#                     output_name=patient_no+"_"+scan_type+"_"+dicom_name.split("-")[1][:-4]+".jpg"
#                     output_path=os.path.join(output_dir_test,output_name)
#                     cv2.imwrite(output_path,image)
#                     flag=1
#     if flag==0:
#         n=len(dicom_list)
#         for i in [int(n/2-1),int(n/2),int(n/2+1)]:
#             dicom_name=dicom_list[i]
#             dicom_path=os.path.join(image_dir,dicom_name)
#             dicom=pydicom.read_file(dicom_path)
#             image=dicom.pixel_array
#             if max_pixel >0:
#                 image=image/max_pixel*255
#                 width, height=image.shape
#                 width_list.append(width)
#                 height_list.append(height)
#                 output_name=patient_no+"_"+scan_type+"_"+dicom_name.split("-")[1][:-4]+".jpg"
#                 output_path=os.path.join(output_dir_test,output_name)
#                 cv2.imwrite(output_path,image)
            
                

# print("width:",np.max(np.array(width_list)),np.min(np.array(width_list)))
# print("height:",np.max(np.array(height_list)),np.min(np.array(height_list)))

In [ ]:
preserve_ratio=0.58
# scan_type_list=["Flair.nii.gz","T1c.nii.gz","T1w.nii.gz","T2w.nii.gz"]
scan_type_list=["T1w.nii.gz"]

# data_dir="/kaggle/input/rsna-test-process/test"
data_dir='/kaggle/working/Final_test'
patient_no_test=os.listdir(data_dir)
print("test numbers:",len(patient_no_test))

width_list=[]
height_list=[]
queue_list=[]

output_dir_test="/kaggle/working/flair2d/test/0"
if not os.path.exists(output_dir_test):
    os.makedirs(output_dir_test)
output_dir_test2="/kaggle/working/flair2d/test/1"
if not os.path.exists(output_dir_test2):
    os.makedirs(output_dir_test2)

for patient_no in patient_no_test:
    for scan_type in scan_type_list:
        image_3d_path=os.path.join(data_dir,patient_no,scan_type)
        image_3d=nib.load(image_3d_path)
        width, height, queue = image_3d.dataobj.shape
        width_list.append(width)
        height_list.append(height)
        queue_list.append(queue)
        for i in range(queue):
            image_2d=image_3d.dataobj[:,:,i]
            max_pixel=np.amax(image_2d)
            if max_pixel >0:
                image_2d=image_2d/max_pixel*255
                if float(np.count_nonzero(image_2d))/image_2d.size>=preserve_ratio:
                    output_name=patient_no+"_"+scan_type.split('.')[0]+"_"+str(i)+".jpg"
                    output_path=os.path.join(output_dir_test,output_name)
#                     print(output_path)
                    cv2.imwrite(output_path,image_2d)
print("width:",np.max(np.array(width_list)),np.min(np.array(width_list)))
print("height:",np.max(np.array(height_list)),np.min(np.array(height_list)))
print("queue:",np.max(np.array(queue_list)),np.min(np.array(queue_list)))

In [ ]:
len(os.listdir(output_dir_test))

### 写/上传inference脚本，进行预测

In [ ]:
os.makedirs("/kaggle/working/preresult")

In [ ]:
# !python /kaggle/input/pytorch-timm/pytorch-image-models/inference_rsna.py  /kaggle/working/flair2d/test  --output_dir  "/kaggle/working/preresult"  --model efficientnetv2_rw_m  --checkpoint  "/kaggle/input/weight/model_best.pth.tar"  --num-classes 2  --img-size 240  --input-size 1 240 240  --batch-size 32
!python /kaggle/input/pytorch-timm/pytorch-image-models/pytorch-image-models/inference_rsna.py  /kaggle/working/flair2d/test  --output_dir  "/kaggle/working/preresult"  --model efficientnetv2_rw_m  --checkpoint  "/kaggle/input/weight/T1w_model_best.pth.tar"  --num-classes 2  --img-size 240  --input-size 3 240 240  --batch-size 32

In [ ]:
os.listdir("/kaggle/working/preresult")

### 集成生成提交结果csv

In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
pre_prob=pd.read_csv("/kaggle/working/preresult/pre_prob.csv",header=None,names=["filename","prob0","prob1"])
print(len(pre_prob))
pre_prob.head()

In [ ]:
prob_dict={}
for i in range(len(pre_prob)):
    filename=pre_prob.iloc[i,0]
    prob=pre_prob.iloc[i,2]
    patient_no=filename[:5]
    if patient_no not in prob_dict.keys():
        prob_dict[patient_no]=[prob]
    else:
        prob_dict[patient_no].append(prob)
print(len(prob_dict))
prob_dict["00001"]

In [ ]:
# sample=pd.read_csv('/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv')
# sample.head()

In [ ]:
# submit_pre=sample
# for i in range(len(sample)):
#     sample_no=sample.loc[i,"BraTS21ID"]
#     patient_no=str(100000+sample_no)[1:]
#     prob_mean=np.mean(np.array(prob_dict[patient_no]))
#     submit_pre.loc[i,"MGMT_value"]=prob_mean

# print(len(submit_pre))
# submit_pre.head()

In [ ]:
column=["BraTS21ID","MGMT_value"]
submit_pre=pd.DataFrame(columns = column)
all_patient=os.listdir("/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/test")
all_patient.sort(key=lambda x: int(x))
for i,patient_no in enumerate(all_patient):
    if patient_no in prob_dict.keys():
        prob_mean=np.mean(np.array(prob_dict[patient_no]))
        submit_pre.loc[i,"BraTS21ID"]=int(patient_no)
        submit_pre.loc[i,"MGMT_value"]=prob_mean
    else:
        submit_pre.loc[i,"BraTS21ID"]=int(patient_no)
        submit_pre.loc[i,"MGMT_value"]=0.1

print(len(submit_pre))
submit_pre.head()

In [ ]:
submit_pre.tail()

In [ ]:
submit_pre.to_csv('./submission.csv',index=False)

In [ ]:
shutil.rmtree("./flair2d")
shutil.rmtree("./preresult")
shutil.rmtree("./Final_test")
shutil.rmtree("./step1_test")